### Fashion MNIST TF DNN

In this notebook we are working on with Fashion Mnist dataset using DNN 

Primary thought behind notebook/notebook series is to build a DNN , CNN and RNN models using tensorflow and see how each alog flares. The codes are structured in such a way that any one with minimal understanding of python and deep learning can uses these notebook/ notebook series, run the blocks and experiment along, gain understanding about DNN CNN and RNN.

Why not Mnist:

    Mnist being too easy and over used - CNN can achive 99.7 % where as DNN can do 97 %. 
    
    Many ML researchers like : Google Brain research scientist and deep learning expert Ian Goodfellow, deep learning expert/Keras author François Chollet. have expersed there concerns on MNIST being overused, and urge the ML community to explore beyound MNIST,  please check there tweets 
    https://twitter.com/goodfellow_ian/status/852591106655043584, https://twitter.com/fchollet/status/852594987527045120 
    
    Further, François Chollet expressed that "MNIST can not represent modern CV tasks", "Many good ideas will not work well on MNIST (e.g. batch norm). Inversely many bad ideas may work on MNIST and not transfer to real CV." 

We start by importing required libraries 

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

c:\python35\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Read the data : The method  used here will be deprecated in later tensorflow release, need to try "Please use alternatives such as official/mnist/dataset.py from tensorflow/models."

In [2]:
fashion_mnist = input_data.read_data_sets('../../dataSets/fashionMnist', one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ../../dataSets/fashionMnist\train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ../../dataSets/fashionMnist\train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting ../../dataSets/fashionMnist\t10k-images-idx3-ubyte.gz
Extracting ../../dataSets/fashionMnist\t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


Initialize some params and hyper params

Num_input : In Mnist each image is represent as 1D vector of length 784, which is 28 * 28 (Heigth of image* Width of the image)
Num_Classes :  Number of ouput types, 'T-shirt/top','Trouser','Pullover','Dress','Coat','Sandal','Shirt','Sneaker','Bag','Ankle boot'. 

Num_Hidden_1, num_hidden_2 : Are the number of units/neurons in the hidden layers.. one of the Params we can play around(feel free to run with different Numbers)

m : Is the number of input instance. 
Num_epochs and minibatch_size : self explonatory.

In [3]:
num_input = 28*28
num_hidden_1 = 128
num_hidden_2 = 128
num_classes = 10
learning_rate = 0.0001

m = 55000
num_epochs = 20
minibatch_size = 64

Begin building TF Graph, with input and output placeholders

InputPlaceholder is tensor of rank two(2D) of shape batch size and number of pixels of input image, 28*28 which will be a vector of length 784. By using None as first D we can feed any number of input instance at given time.

OutputPlaceholder : is again a tensor of rank two, with shape batch size and number of output classes, which is 10.

In [4]:
X_placeholder = tf.placeholder(tf.float32, [None, num_input], name='X')
Y_placeholder = tf.placeholder(tf.float32, [None, num_classes], name='Y')

Hidden and Output Layers Weights and bias - Initialize weights for all the  remaining layers, apart from Input layer(X_placeholder), with proper dimensions. "tf.get_variable" is high level API of "tf.Variable", tensorflow suggests to use former. We have initialzed the weights as random normal, bias as zero. We can play with other initializer as well like random uniform, truncated normal .. I suggest using "Xavier_initializer" which is in contib for this version of tensorflow. 

In [5]:
W1 = tf.get_variable("W1", [num_input, num_hidden_1], initializer=tf.initializers.random_normal(seed=12))
b1 = tf.get_variable("b1", [1, num_hidden_1], initializer=tf.zeros_initializer())
                
W2 = tf.get_variable("W2", [num_hidden_1, num_hidden_2], initializer=tf.initializers.random_normal(seed=12))
b2 = tf.get_variable("b2", [1, num_hidden_2], initializer=tf.zeros_initializer())
                
W3 = tf.get_variable("W3", [num_hidden_2, num_classes], initializer=tf.initializers.random_normal(seed=12))
b3 = tf.get_variable("b3", [1, num_classes], initializer=tf.zeros_initializer())

Forward pass - 
Now computate output of each layer, which is multiply input with the corresponding weight and adding respective bias, finally running a non linear activation (Relu) over the results.

In [6]:
Z1 = tf.add(tf.matmul(X_placeholder, W1), b1)     
A1 = tf.nn.relu(Z1)                  
Z2 = tf.add(tf.matmul(A1, W2), b2)    
A2 = tf.nn.relu(Z2)                  
final_layer_result = tf.add(tf.matmul(A2, W3), b3)

We train models with an objective to reduce the loss/error/cost. We want to minimize this error to make our model more accurate.To calculate the cost we use cross entropy, loss/error/ cost measures how inefficient our predictions are for describing the truth. We train our model using Adam optimizer, encourage you to experiment with RMP prop 

Because TensorFlow knows the entire graph of your computations, it can automatically use the backpropagation algorithm to efficiently determine how your variables affect the loss you ask it to minimize. Then it can apply your choice of optimization algorithm to modify the variables and reduce the loss.

Next, We ask TensorFlow to minimize cross_entropy using the gradient descent algorithm with a learning rate of 0.5.

In [7]:
#cost_cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y), reduction_indices=[1]))

In [8]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=final_layer_result, labels=Y_placeholder))
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

In [9]:
accuracy = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(final_layer_result, 1), tf.argmax(Y_placeholder, 1)), tf.float32))

rest is requisite, initialize the variables,  we creat/ initiate a session , run the model on #ofEpochs, in predifined mini batches.

In [10]:
init = tf.global_variables_initializer()

In [11]:
with tf.Session() as sess:
    
    costs = []
    # Run initialization
    sess.run(init)
            
    for epoch in range(num_epochs):

        printLoss = True
        train = fashion_mnist.train
        Xtest = fashion_mnist.test.images
        ytest = fashion_mnist.test.labels
        epoch_loss = 0.
        num_minibatches = int(m / minibatch_size)
        
        for i in range(num_minibatches):
            minibatch_X, minibatch_Y = train.next_batch(minibatch_size)
            _, minibatch_loss = sess.run([optimizer, cost], feed_dict={X_placeholder: minibatch_X, Y_placeholder: minibatch_Y})
            epoch_loss += minibatch_loss 
        costs.append(epoch_loss / num_minibatches)
            
        print ("Testing data accuracy : {accuracy}".format(
            accuracy = sess.run([accuracy], feed_dict={X_placeholder: Xtest, Y_placeholder: ytest})))

Testing data accuracy : [0.5338]
Testing data accuracy : [0.6321]
Testing data accuracy : [0.6702]
Testing data accuracy : [0.6931]
Testing data accuracy : [0.7085]
Testing data accuracy : [0.7166]
Testing data accuracy : [0.7263]
Testing data accuracy : [0.7336]
Testing data accuracy : [0.7406]
Testing data accuracy : [0.7472]
Testing data accuracy : [0.7523]
Testing data accuracy : [0.756]
Testing data accuracy : [0.7584]
Testing data accuracy : [0.7604]
Testing data accuracy : [0.7634]
Testing data accuracy : [0.7652]
Testing data accuracy : [0.7671]
Testing data accuracy : [0.7666]
Testing data accuracy : [0.769]
Testing data accuracy : [0.7698]


### Using Tensorflow API (Dense layer) 

In [12]:
import numpy as np
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [13]:
mnist = input_data.read_data_sets('../../dataSets/fashionMnist', one_hot=True)

Extracting ../../dataSets/fashionMnist\train-images-idx3-ubyte.gz
Extracting ../../dataSets/fashionMnist\train-labels-idx1-ubyte.gz
Extracting ../../dataSets/fashionMnist\t10k-images-idx3-ubyte.gz
Extracting ../../dataSets/fashionMnist\t10k-labels-idx1-ubyte.gz


In [14]:
Xtrain = mnist.train.images
ytrain = mnist.train.labels
Xtest = mnist.test.images
ytest = mnist.test.labels

In [15]:
num_input = 28 * 28
num_classes = 10
num_hidden_1 = 128
num_hidden_2 = 128
num_epochs = 20
minibatch_size = 64

In [16]:
X_placeholder = tf.placeholder(tf.float32, [None, num_input], name="X")
Y_placeholder = tf.placeholder(tf.float32, [None, num_classes], name="Y")

In [17]:
hidden_1 = tf.layers.dense(X_placeholder, num_hidden_1, activation=tf.nn.relu, use_bias=True,
                         kernel_initializer=tf.truncated_normal_initializer(stddev=num_input**-0.5))

In [18]:
hidden_2 = tf.layers.dense(hidden_1, num_hidden_2, activation=tf.nn.relu, use_bias=True,
                         kernel_initializer=tf.truncated_normal_initializer(stddev=num_hidden_1**-0.5))

In [19]:
y = tf.layers.dense(hidden_2, num_classes, activation=None, use_bias=True,
                    kernel_initializer=tf.truncated_normal_initializer(stddev=num_hidden_2**-0.5))

In [20]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=y, labels=Y_placeholder))
accuracy = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(y, 1), tf.argmax(Y_placeholder, 1)),
                                  tf.float32))

In [21]:
#optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(cost)
optimizer = tf.train.AdamOptimizer(0.0001).minimize(cost)
idx = range(Xtrain.shape[0])
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(num_epochs):
        for j in range(0, len(idx), minibatch_size):
            sess.run(optimizer, feed_dict={X_placeholder: Xtrain[idx[j:j+minibatch_size]],
                                     Y_placeholder: ytrain[idx[j:j+minibatch_size]]})

        print (sess.run([ accuracy], feed_dict={X_placeholder: Xtest, Y_placeholder: ytest}))

[0.8036]
[0.8277]
[0.838]
[0.8423]
[0.8481]
[0.8529]
[0.8563]
[0.8582]
[0.8604]
[0.8624]
[0.865]
[0.8659]
[0.8672]
[0.8684]
[0.8695]
[0.87]
[0.8707]
[0.871]
[0.8725]
[0.8736]
